In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cs3120-ames-home-prices/sample_submission.csv
/kaggle/input/cs3120-ames-home-prices/data_description.txt
/kaggle/input/cs3120-ames-home-prices/train.csv
/kaggle/input/cs3120-ames-home-prices/test.csv


Above is the standard code cell that Kaggle will include anytime you are in a competition and click on, "New Notebook". As you can see, the idea is to list out the (data) files that are a part of this competition. The general process you'll want to follow is:

1. Load `train.csv`
2. Split train.csv into your own train and test (or validation) dataset splits
3. Decide which features you want to include in your model (this may also include your own *Feature Engineering* steps
3. Train a model using your own train split, tune the model with your test split
4. After you're satisfied with your model, load the `test.csv` file, and then make predictions using your trained model
5. Save your predictions in a file, e.g. submission.csv, then submit these predictions to the competition (see [Submitting Predictions](https://www.kaggle.com/docs/competitions#submitting-predictions) for more details)

Note: You will likely want to interate between steps 2. and 5. several times, since you can always make a submission, and see how accurate your model is on the Kaggle competition test dataset (which is different from your test split), and then go back try something different with steps 2. and 3. again. 

Now, we'll do step 1., and load the train dataset. 


In [2]:
df = pd.read_csv('/kaggle/input/cs3120-ames-home-prices/train.csv')

We'll now slit the training dataset into separate train and test splits. However, remember that you could instead use a Cross-Validation approach, which case you might not need this. 

Before doing so, you'll likely want to create an `X`, which will hold all your features (or predictors, or independent variables, etc.), and a `y`, which will be the outcome (or target, or dependent variable, etc.). You will definitely want to use more than just one feature, as is done below. 

You may want to go back to the last Kaggle course you did, or the notebook from [Assign 4](https://msudenver.instructure.com/courses/98353/assignments/1311195).

In [3]:
from sklearn.model_selection import train_test_split

X = df[['Overall Qual','Lot Area','Gr Liv Area','Garage Cars','Fireplaces','Year Built',
        'Year Remod/Add']]
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)


In [4]:
# Remove any missing values (you may want to do something different here too)


In [5]:
# Train a model
from sklearn.ensemble import HistGradientBoostingRegressor

best_params = {'learning_rate': 0.1, 'n_estimators': 101}
model = HistGradientBoostingRegressor(random_state=2)
model.fit(X_train, y_train)

HistGradientBoostingRegressor(random_state=2)

In [6]:
# Tune your model (try different hyperparameters, try different features, etc.)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

y_pred = model.predict(X_test)
print("Model Performance Metrics:")
print(f"Mean Absolute Error (MAE): {mean_absolute_error(y_test, y_pred):.2f}")
print(f"Root Mean Squared Error (RMSE): {mean_squared_error(y_test, y_pred, squared=False):.2f}")
print(f"R^2 Score: {r2_score(y_test, y_pred):.2f}")

# print("Total Missing Values in X_sub:", X_sub.isnull().sum().sum())

# Use grid search to find the best hyperparameters for the Gradient Boosting Regressor
# param_grid = {
#     'n_estimators': list(range(1, 402, 50)),
#     'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.5]
# }

# model_gb = GradientBoostingRegressor(random_state=2)
# grid_search = GridSearchCV(model_gb, param_grid, cv=5, scoring='r2')
# grid_search.fit(X, y)

# print(f"** Best R2 = {grid_search.best_score_:1.2E} **")
# print(f"** Best Parameters = {grid_search.best_params_} **")


# Use cross-validation to get a stable R² estimate
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print(f"Cross-Validation R^2 Scores: {cv_scores}")
print(f"Average Cross-Validation R^2: {cv_scores.mean():.2f}")

Model Performance Metrics:
Mean Absolute Error (MAE): 22566.41
Root Mean Squared Error (RMSE): 36906.48
R^2 Score: 0.84
Cross-Validation R^2 Scores: [0.85504683 0.89695721 0.85501502 0.81268654 0.78228346]
Average Cross-Validation R^2: 0.84


Once you've trained your model, you can then make predictions and submit them as a solution to the competition. You first need to load the separate `test.csv` dataset, then use your model to make predictions for this data. 

In [7]:
df_sub = pd.read_csv('/kaggle/input/cs3120-ames-home-prices/test.csv')

X_sub = df_sub[['Overall Qual','Lot Area','Gr Liv Area','Garage Cars','Fireplaces','Year Built',
        'Year Remod/Add']]


In [8]:
df_sub['SalePrice'] = model.predict(X_sub)

You likely want to quickly check your predictions to see if they make sense. Note that you don't have the true `y` values, so you can't compare them, but you can at least check that they make sense. 

In [9]:
df_sub[['Id', 'SalePrice','Overall Qual','Lot Area','Gr Liv Area','Garage Cars','Fireplaces','Year Built','Year Remod/Add']].head()

,Id,SalePrice,Overall Qual,Lot Area,Gr Liv Area,Garage Cars,Fireplaces,Year Built,Year Remod/Add
0,1000,169649.735062,7,10300,1710,2.0,0,1921,1950
1,1001,128346.514149,5,8750,1202,1.0,0,1956,1956
2,1002,99420.867693,5,9045,2256,0.0,0,1910,1950
3,1003,372362.770867,9,13514,1808,3.0,1,2008,2008
4,1004,137351.042828,6,10454,1105,1.0,0,1957,1957


Although it doesn't seem that important, the following step of saving your test set predictions is necessary in order to make a submission to the competition. 

In [10]:
df_sub[['Id', 'SalePrice']].to_csv('/kaggle/working/submission.csv', index=False)